In [1]:
# https://thebook.io/080228/part04/ch13/

In [ ]:
# 음파 탐지기를 쏜 후 그 결과를 데이터로 정리
# 정확도는 몇 퍼센트(%)였을까
# 지금 딥러닝의 효시가 된 중요한 알고리즘이 발표 : 역전파
# 이제 우리는 가정용 컴퓨터와 각종 라이브러리를 통해 
# 어렵지 않게 딥러닝을 구현하는 시대
# ‘과적합(overfitting)’에 관한 내용도 다루어 봄

In [2]:
# 먼저 필요한것 import
# 참고로 책은 keras를 사용한 옛날 소스라 조금 이상할것

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import tensorflow as tf

In [4]:
# seed 값 설정
np.random.seed(3)
tf.random.set_seed(3)

In [5]:
# 먼저 데이터를 확인
df = pd.read_csv('data/sonar.csv', header=None)

In [7]:
print(df.head())

       0       1       2       3       4       5       6       7       8   \
0  0.0200  0.0371  0.0428  0.0207  0.0954  0.0986  0.1539  0.1601  0.3109   
1  0.0453  0.0523  0.0843  0.0689  0.1183  0.2583  0.2156  0.3481  0.3337   
2  0.0262  0.0582  0.1099  0.1083  0.0974  0.2280  0.2431  0.3771  0.5598   
3  0.0100  0.0171  0.0623  0.0205  0.0205  0.0368  0.1098  0.1276  0.0598   
4  0.0762  0.0666  0.0481  0.0394  0.0590  0.0649  0.1209  0.2467  0.3564   

       9   ...      51      52      53      54      55      56      57  \
0  0.2111  ...  0.0027  0.0065  0.0159  0.0072  0.0167  0.0180  0.0084   
1  0.2872  ...  0.0084  0.0089  0.0048  0.0094  0.0191  0.0140  0.0049   
2  0.6194  ...  0.0232  0.0166  0.0095  0.0180  0.0244  0.0316  0.0164   
3  0.1264  ...  0.0121  0.0036  0.0150  0.0085  0.0073  0.0050  0.0044   
4  0.4459  ...  0.0031  0.0054  0.0105  0.0110  0.0015  0.0072  0.0048   

       58      59  60  
0  0.0090  0.0032   R  
1  0.0052  0.0044   R  
2  0.0095  0.0078   

In [8]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 61 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       208 non-null    float64
 1   1       208 non-null    float64
 2   2       208 non-null    float64
 3   3       208 non-null    float64
 4   4       208 non-null    float64
 5   5       208 non-null    float64
 6   6       208 non-null    float64
 7   7       208 non-null    float64
 8   8       208 non-null    float64
 9   9       208 non-null    float64
 10  10      208 non-null    float64
 11  11      208 non-null    float64
 12  12      208 non-null    float64
 13  13      208 non-null    float64
 14  14      208 non-null    float64
 15  15      208 non-null    float64
 16  16      208 non-null    float64
 17  17      208 non-null    float64
 18  18      208 non-null    float64
 19  19      208 non-null    float64
 20  20      208 non-null    float64
 21  21      208 non-null    float64
 22  22

In [9]:
# 맨 마지막 타입이 object

In [10]:
print(df.tail())

         0       1       2       3       4       5       6       7       8   \
203  0.0187  0.0346  0.0168  0.0177  0.0393  0.1630  0.2028  0.1694  0.2328   
204  0.0323  0.0101  0.0298  0.0564  0.0760  0.0958  0.0990  0.1018  0.1030   
205  0.0522  0.0437  0.0180  0.0292  0.0351  0.1171  0.1257  0.1178  0.1258   
206  0.0303  0.0353  0.0490  0.0608  0.0167  0.1354  0.1465  0.1123  0.1945   
207  0.0260  0.0363  0.0136  0.0272  0.0214  0.0338  0.0655  0.1400  0.1843   

         9   ...      51      52      53      54      55      56      57  \
203  0.2684  ...  0.0116  0.0098  0.0199  0.0033  0.0101  0.0065  0.0115   
204  0.2154  ...  0.0061  0.0093  0.0135  0.0063  0.0063  0.0034  0.0032   
205  0.2529  ...  0.0160  0.0029  0.0051  0.0062  0.0089  0.0140  0.0138   
206  0.2354  ...  0.0086  0.0046  0.0126  0.0036  0.0035  0.0034  0.0079   
207  0.2354  ...  0.0146  0.0129  0.0047  0.0039  0.0061  0.0040  0.0036   

         58      59  60  
203  0.0193  0.0157   M  
204  0.0062  0.0

In [11]:
dataset = df.values  # <- 문제는 이러면 data를 싸그리 가져옴
# numpy 배열은 속도를 최우선으로 두기 때문에, 반드시 1가지 타입으로만 가져옴
# 위에 float과 object가 저렇게 있으면, 싸그리 float으로 가져옴

# X = dataset[:,0:60]
# 이런 식으로 가져오면 마지막의 object때문에 가져올때 에러남 -> .astype(float)으로 바꿔(통일)줘야함

X = dataset[:,0:60].astype(float)
Y_obj = dataset[:,60]

In [12]:
# 문자열 변환
e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)

# 어제랑은 다르게
# 소프트맥스 안쓸거니까, 원핫인코딩 필요 없음

In [13]:
# 모델 설정
model = Sequential()
model.add(Dense(24, input_dim=60, activation='relu'))  # input은 60개였음, activation은 relu로 잡음
model.add(Dense(10, activation='relu'))  # 출력은 10개.
model.add(Dense(1, activation='sigmoid'))  # 출력은 1 할 것. activation은 sigmoid

In [14]:
# 가중치 계산
# 방법 까먹음..

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 24)                1464      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                250       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 1,725
Trainable params: 1,725
Non-trainable params: 0
_________________________________________________________________


In [16]:
# 모델 컴파일
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
# 손실은 mean~ 을 줌. optimizer는 adam <- 잘 모르면 그냥 adma
# metrics는 accuracy

In [17]:
# 모델 실행
model.fit(X, Y, epochs=200, batch_size=5)

Epoch 1/200
42/42 [==============================] - 0s 924us/step - loss: 0.2432 - accuracy: 0.5865
Epoch 2/200
42/42 [==============================] - 0s 810us/step - loss: 0.2288 - accuracy: 0.6394
Epoch 3/200
42/42 [==============================] - 0s 865us/step - loss: 0.2187 - accuracy: 0.6490
Epoch 4/200
42/42 [==============================] - 0s 900us/step - loss: 0.2031 - accuracy: 0.7260
Epoch 5/200
42/42 [==============================] - 0s 900us/step - loss: 0.1893 - accuracy: 0.7692
Epoch 6/200
42/42 [==============================] - 0s 778us/step - loss: 0.1802 - accuracy: 0.7596
Epoch 7/200
42/42 [==============================] - 0s 827us/step - loss: 0.1677 - accuracy: 0.7740
Epoch 8/200
42/42 [==============================] - 0s 803us/step - loss: 0.1590 - accuracy: 0.7837
Epoch 9/200
42/42 [==============================] - 0s 876us/step - loss: 0.1558 - accuracy: 0.7885
Epoch 10/200
42/42 [==============================] - 0s 827us/step - loss: 0.1483 - accura

In [18]:
# 결과 출력
print("\n Accuracy: %.4f" % (model.evaluate(X, Y)[1]))  # evaluate 로 평가

7/7 [==============================] - 0s 833us/step - loss: 0.0054 - accuracy: 0.9952

 Accuracy: 0.9952
